<a href="https://colab.research.google.com/github/carollimaxp/data_science_alura/blob/main/DS_Bootcamp_M%C3%B3dulo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré Aula 1


1.   Imports
2.   Transformação dos milhares



Imports para o Módulo 2

In [ ]:
#Imports - Módulo 1
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties
import numpy as np

Formatação dos Reais em milhões e bilhões

In [ ]:
# url https://dfrieds.com/data-visualizations/when-use-log-scale.html

def reformat_large_tick_values(tick_val, pos):
    """
    Turns large tick values (in the billions, millions and thousands) such as 4500 into 4.5K and also appropriately turns 4000 into 4K (no zero after the decimal).
    """
    if tick_val >= 1000000000:
        val = round(tick_val/1000000000, 1)
        new_tick_format = '{:}B'.format(val)
    elif tick_val >= 1000000:
        val = round(tick_val/1000000, 1)
        new_tick_format = '{:}M'.format(val)
    elif tick_val >= 1000:
        val = round(tick_val/1000, 1)
        new_tick_format = '{:}K'.format(val)
    elif tick_val < 1000:
        new_tick_format = round(tick_val, 1)
    else:
        new_tick_format = tick_val

    # make new_tick_format into a string value
    new_tick_format = str(new_tick_format)
    
    # code below will keep 4.5M as is but change values such as 4.0M to 4M since that zero after the decimal isn't needed
    index_of_decimal = new_tick_format.find(".")
    
    if index_of_decimal != -1:
        value_after_decimal = new_tick_format[index_of_decimal+1]
        if value_after_decimal == "0":
            # remove the 0 after the decimal point since it's not needed
            new_tick_format = new_tick_format[0:index_of_decimal] + new_tick_format[index_of_decimal+2:]
            
    return new_tick_format

# Aula 1

### Leitura cvs

In [ ]:
dados = pd.read_csv("https://raw.githubusercontent.com/carollimaxp/data_science_alura/main/Dados/A160324189_28_143_208.csv", 
                    encoding="ISO-8859-1", 
                    skiprows = 3,
                    delimiter=";",
                    skipfooter = 12,
                    thousands=".",
                    decimal=",",
                    na_values = "-")
dados.head()

In [ ]:
### ...